In [7]:
import pandas as pd
import numpy as np
import math
import math
from Util import *
from CLT_class import CLT
from __future__ import print_function

datasetList = ["nltcs","plants","accidents"]
data = {}
for i in datasetList:
    data[i+".ts.data"] = Util.load_dataset("datasets\\dataset\\"+i+".ts.data")
    data[i+".test.data"] = Util.load_dataset("datasets\\dataset\\"+i+".test.data")
    data[i+".valid.data"] = Util.load_dataset("datasets\\dataset\\"+i+".valid.data")

    

In [2]:
class MIXTURE_CLT():
   
    def __init__(self):
        self.n_components = 0 # number of components
        self.mixture_probs = None # mixture probabilities
        self.clt_list =[]   # List of Tree Bayesian networks

    '''
        Learn Mixtures of Trees using the EM algorithm.
    '''
    def learn(self, dataset, n_components=2, max_iter=50, epsilon=format(1e-1, '.6f')):
        # For each component and each data point, we have a weight
        weights=np.random.random((n_components , dataset.shape[0]))
        self.mixture_probs = np.zeros(n_components)
 
        # Randomly initialize the chow-liu trees and the mixture probabilities
        # Your code for random initialization goes here
 
        for i in range(n_components):
            self.clt_list.append(CLT())
            self.clt_list[i].learn(dataset)
 
        a = np.random.rand(n_components)
        for i in range(len(a)):
            self.mixture_probs[i] = a[i] / np.sum(a)  

        like = 0.0
        old_like = 999
        prob = np.zeros((n_components , dataset.shape[0]))


        for itr in range(max_iter):
           
            print("Iteration - ",itr)
            print("N components - ",n_components)
           
            #E-step: Complete the dataset to yield a weighted dataset
            # We store the weights in an array weights[ncomponents,number of points]
            #Your code for E-step here
            
            if abs(old_like - like) <= epsilon:
                break


            for i in range(n_components):
                for j in range(dataset.shape[0]):
                    prob[i][j] = self.clt_list[i].getProb(dataset[j])
                weights[i] = np.multiply(self.mixture_probs[i], prob[i])/np.sum(np.multiply(self.mixture_probs[i], prob[i]))
                # print(weights.shape)
        
            # # M-step: Update the Chow-Liu Trees and the mixture probabilities
            # #Your code for M-Step here
            for i in range(n_components):
                self.clt_list[i].update(dataset, weights[i])
                self.mixture_probs[i] = weights[i].sum()/len(weights[i])
                
            old_like = like
            like = self.computeLL(dataset , n_components) / dataset.shape[0]
            print(like)

         
           
    # """
    #     Compute the log-likelihood score of the dataset
    # """
    def computeLL(self, dataset , n_components):
        ll = 0.0
        likelihood=0.0

        # Write your code below to compute likelihood of data
        #   Hint:   Likelihood of a data point "x" is sum_{c} P(c) T(x|c)
        #           where P(c) is mixture_prob of cth component and T(x|c) is the probability w.r.t. chow-liu tree at c
        #           To compute T(x|c) you can use the function given in class CLT
        for i in range (dataset.shape[0]):
            for j in range(n_components):
                likelihood=likelihood + self.mixture_probs[j]*self.clt_list[j].getProb(dataset[i])
            ll=ll+ np.log(likelihood)


        return ll
   
   

In [3]:
#MCLT
kval = {}
items = list(data.keys())
# mix_clt=MIXTURE_CLT.MIXTURE_CLT()
ncomponents=10 #number of components
max_iter=50 #max number of iterations for EM
epsilon=1e-2 #converge if the difference in the log-likelihods between two iterations is smaller 1e-1


i = 0

kvals = [2,5,10,20]
while i < 30:
    kMixCLT = [MIXTURE_CLT() for p in kvals]
    p = 0
    temp = []
    print("Running for dataset - ", items[i])
    for k in kvals:
        t = i 
        # print([items[t]])
        kMixCLT[p].learn(data[items[t]],k,max_iter,epsilon)
        t += 2
        x = kMixCLT[p].computeLL(data[items[t]] , k)/data[items[t]].shape[0]
        print("LL for ", k , " - ", x  )
        temp.append(x)
        p += 1
        
    kval[items[i]] = [ kvals[temp.index(max(temp))] , max(temp)] 

    print(temp)
    print(kval)
    i += 3

Running for dataset -  nltcs.ts.data
Iteration -  0
N components -  2
-2.510874528149324
Iteration -  1
N components -  2
-2.105008024515309
Iteration -  2
N components -  2
-2.031459553492355
Iteration -  3
N components -  2
-2.013390526785948
Iteration -  4
N components -  2
-2.008416748426295
Iteration -  5
N components -  2
LL for  2  -  -4.0584007557343975
Iteration -  0
N components -  5
-1.5945837962750757
Iteration -  1
N components -  5
-1.1887172926412217
Iteration -  2
N components -  5
-1.115168821618279
Iteration -  3
N components -  5
-1.097099794911924
Iteration -  4
N components -  5
-1.0921260165521847
Iteration -  5
N components -  5
LL for  5  -  -3.142110023860273
Iteration -  0
N components -  10
-0.9014366157151267
Iteration -  1
N components -  10
-0.495570112081406
Iteration -  2
N components -  10
-0.4220216410583785
Iteration -  3
N components -  10
-0.40395261435211627
Iteration -  4
N components -  10
-0.398978835992392
Iteration -  5
N components -  10
LL f

IndexError: list index out of range

In [4]:
items = list(data.keys())
ncomponents=20
max_iter=50 
epsilon=1e-1 
i = 0
temp = []

for i in range(5):
    clt = MIXTURE_CLT()
    print("Testing for dataset - plants")
    clt.learn(data["plants.ts.data"],20,max_iter,epsilon)
    x = clt.computeLL( data["plants.test.data"] , k)/data["plants.test.data"].shape[0]
    print("LL for testing - " , x  )
    temp.append(x)
        

    print(temp)


Testing for dataset - plants
Iteration -  0
N components -  20
-2.498699005667827
Iteration -  1
N components -  20
-1.8673372743507264
Iteration -  2
N components -  20
-1.711673819386216
Iteration -  3
N components -  20
-1.6607001701541884
Iteration -  4
N components -  20
LL for testing -  -3.716719916883191
[-3.716719916883191]
Testing for dataset - plants
Iteration -  0
N components -  20
-2.498699005667827
Iteration -  1
N components -  20
-1.8673372743507264
Iteration -  2
N components -  20
-1.711673819386216
Iteration -  3
N components -  20
-1.6607001701541884
Iteration -  4
N components -  20
LL for testing -  -3.716719916883191
[-3.716719916883191, -3.716719916883191]
Testing for dataset - plants
Iteration -  0
N components -  20
-2.498699005667827
Iteration -  1
N components -  20
-1.8673372743507264
Iteration -  2
N components -  20
-1.711673819386216
Iteration -  3
N components -  20
-1.6607001701541884
Iteration -  4
N components -  20
LL for testing -  -3.716719916883

In [9]:
items = list(data.keys())
ncomponents=20
max_iter=50 
epsilon=1e-1 
i = 0
temp = []

for i in range(5):
    clt = MIXTURE_CLT()
    print("Testing for dataset - accidents")
    clt.learn(data["accidents.ts.data"],20,max_iter,epsilon)
    x = clt.computeLL( data["accidents.test.data"] , 20)/data["accidents.test.data"].shape[0]
    print("LL for testing - " , x  )
    temp.append(x)
        

    print(temp)


Testing for dataset - accidents
Iteration -  0
N components -  20
-13.235720513753012
Iteration -  1
N components -  20
-7.312002140639947
Iteration -  2
N components -  20
-7.4537986651319414
Iteration -  3
N components -  20
-7.490548143822814
Iteration -  4
N components -  20
LL for testing -  -40.25831983921568
[-40.25831983921568]
Testing for dataset - accidents
Iteration -  0
N components -  20
-13.235720513753012
Iteration -  1
N components -  20
-7.312002140639947
Iteration -  2
N components -  20
-7.45449915102762
Iteration -  3
N components -  20
-7.488501497220625
Iteration -  4
N components -  20
LL for testing -  -40.219937234696914
[-40.25831983921568, -40.219937234696914]
Testing for dataset - accidents
Iteration -  0
N components -  20
-13.23572051375301
Iteration -  1
N components -  20
-7.312002140639947
Iteration -  2
N components -  20
-7.454901258991249
Iteration -  3
N components -  20
-7.491239864571279
Iteration -  4
N components -  20
LL for testing -  -40.2134